In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
# df=pd.read_csv("/DATA/jimson/Rishi/Priyadrasta/AutoMM/augmented_2nd_Floor_Dataset_rescored_new.csv")

In [ ]:
# df.shape

In [ ]:
# df.columns

In [ ]:
# df['key']=df['Var1'].apply(lambda x: int(x.split("_")[2]))
# df['key'].value_counts()

In [ ]:
# df=df.reindex(columns=['Var1', 'numWhitePixels_left', 'numWhitePixels_right', 'diff_pixel',
#        'left_area', 'right_area', 'diff_area', 'ssim_left_right',
#        'peaksnr_left_right', 'mse_left_right', 'abs_left_right', 'cosine_lr', 'image',
#        'label'])

In [ ]:
# df.head(10)

In [ ]:
# df['label'].value_counts()/4

In [ ]:
# df['label'].value_counts().plot.pie(autopct='%.2f')

In [ ]:
# from PIL import Image
# for i in range(831,833):
#     example_image = df.iloc[i]['image']
#     # print(example_image)

#     pil_img = Image.open(example_image)
#     # display(pil_img)
#     # pil_img = pil_img.resize((1024,1024),resample=Image.LANCZOS)
#     print(pil_img.size)

In [ ]:
#split_loc=1372
#df.iloc[split_loc]

In [ ]:
# df['label']=df['label'].apply(lambda x: int(x>2))

In [ ]:
df_train=pd.read_csv("second_train_binary.csv")
df_test=pd.read_csv("second_test_binary.csv")

In [ ]:
from sklearn.model_selection import train_test_split
#df_train,df_test=df.iloc[:split_loc,:],df.iloc[split_loc:,:]
# df_train, df_test = train_test_split(df, test_size=0.3)
df_train.shape,df_test.shape

In [ ]:
# df_test.head(3)

In [ ]:
# df_train.columns

In [ ]:
# df_train['key']=df_train['Var1'].apply(lambda x: int(x.split("_")[2]))
# df_train['key'].value_counts()

In [ ]:
print(df_train['label'].value_counts(),df_test['label'].value_counts(),sep="\n")

In [ ]:
from autogluon.multimodal import MultiModalPredictor

In [ ]:
predictor = MultiModalPredictor.load("/DATA/jimson/Rishi/Priyadrasta/AutoMM/AutogluonModels/ag-20230429_143620")

In [ ]:
scores = predictor.evaluate(df_test, metrics=["roc_auc"])
scores

In [ ]:
scores = predictor.evaluate(df_test,metrics=['accuracy'])
scores

In [ ]:
prediction = predictor.predict(df_test.drop(columns='label'))
prediction

In [ ]:
from sklearn.metrics import confusion_matrix
conf_matrix= confusion_matrix(df_test['label'].to_numpy(),prediction.to_numpy())
conf_matrix

In [ ]:
FP = conf_matrix.sum(axis=0) - np.diag(conf_matrix) 
FN = conf_matrix.sum(axis=1) - np.diag(conf_matrix)
TP = np.diag(conf_matrix)
TN = conf_matrix.sum() - (FP + FN + TP)
FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy for each class
ACC = (TP+TN)/(TP+FP+FN+TN)
print("class-wise  Accuracy :",ACC)
print("class-wise  Precision :",PPV)
print("class-wise  Sensitivity :",TPR)
print("class-wise  Specificity :",TNR)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(df_test['label'].to_numpy(), prediction.to_numpy()))

In [ ]:
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt
 
fig, ax = plot_confusion_matrix(conf_mat=conf_matrix, figsize=(4, 4), cmap=plt.cm.Blues)
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix', fontsize=18)
plt.show()

In [ ]:
df_test.shape

In [ ]:
# # predictor.save('predictor_multiclass_9157')
# loaded_predictor = MultiModalPredictor.load('predictor_multiclass_9157')
# scores2 = loaded_predictor.evaluate(df_test, metrics=["accuracy"])
# scores2

In [ ]:
df_test.shape

In [ ]:
df_test['label'].value_counts()

In [ ]:
prediction.value_counts()

In [ ]:
probas = predictor.predict_proba(df_test.drop(columns='label'))
probas.shape

In [ ]:
probas_arr=probas.values
print(probas_arr.shape)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
false_positive_rate, true_positive_rate, threshold = roc_curve(df_test['label'], prediction)
print('roc_auc_score : ', roc_auc_score(df_test['label'], prediction))
print('auc_score : ', auc(false_positive_rate, true_positive_rate))
plt.title('ROC curve  (Binary)')
plt.plot(false_positive_rate, true_positive_rate,'tab:orange', marker='o',
label='(area = {0:0.2f})'.format(auc(false_positive_rate, true_positive_rate)) ,ms=5)
# plt.plot([0, 1], ls="--")
# plt.plot([0, 0], [1, 0] , c=".7")
# plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend()
plt.show()

In [ ]:
np.save("/DATA/jimson/Rishi/Priyadrasta/papercode/second floor/binary/efficientnet/fprmacro_binary.npy", false_positive_rate)
np.save("/DATA/jimson/Rishi/Priyadrasta/papercode/second floor/binary/efficientnet/tprmacro_binary.npy",  true_positive_rate)

In [ ]:
# df['prediction']=prediction
# df.drop(columns=["label"],inplace=True)
# df.to_csv("3rd_Floor_Predicted.csv",index=False)